In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("feature-extraction", model="intfloat/e5-mistral-7b-instruct")

/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 629/629 [00:00<00:00, 416kB/s]
model.safetensors.index.json: 100%|██████████| 23.3k/23.3k [00:00<00:00, 14.4MB/s]
model-00001-of-00002.safetensors:  51%|█████     | 5.04G/9.94G [02:43<02:39, 30.8MB/s]


KeyboardInterrupt: 

In [ ]:
pipe("Hello")

In [2]:
import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'


# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'
input_texts = [get_detailed_instruct(task, 'how much protein should a female eat'),
               get_detailed_instruct(task, 'summit define'),
               "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."]

tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-mistral-7b-instruct')
model = AutoModel.from_pretrained('intfloat/e5-mistral-7b-instruct')

max_length = 4096
# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=max_length - 1, return_attention_mask=False, padding=False, truncation=True)
# append eos_token_id to every input_ids
batch_dict['input_ids'] = [input_ids + [tokenizer.eos_token_id] for input_ids in batch_dict['input_ids']]
batch_dict = tokenizer.pad(batch_dict, padding=True, return_attention_mask=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


tokenizer_config.json: 100%|██████████| 956/956 [00:00<00:00, 3.47MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 4.50MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 8.83MB/s]
added_tokens.json: 100%|██████████| 42.0/42.0 [00:00<00:00, 87.1kB/s]
special_tokens_map.json: 100%|██████████| 168/168 [00:00<00:00, 335kB/s]
Trying to resume download...




















model-00001-of-00002.safetensors: 100%|██████████| 9.94G/9.94G [00:04<00:00, 42.4MB/s]
model-00001-of-00002.safetensors:  98%|█████████▊| 9.74G/9.94G [19:49<00:24, 8.19MB/s]
Trying to resume download...















































































































model-00002-of-00002.safetensors:  27%|██▋       | 1.14G/4.28G [01:30<04:07, 12.7MB/s]
model-00002-of-00002.safetensors:   0%|          | 0.00/4.28G [04:15<?, ?B/s]


KeyboardInterrupt: 